In [1]:
import tensorflow as tf


@tf.function
def comp_classification_loss(y_true, y_pred):
    return -1/num_classes * tf.reduce_sum(y_true * tf.math.log(y_pred + 1e-9) + (1 - y_true) * tf.math.log(1 - y_pred + 1e-9))

@tf.function
def comp_regression_loss(y_true, y_pred):
    return 1/num_classes * tf.reduce_sum(tf.square(y_true - y_pred))

@tf.function
def custom_loss(y_true, y_pred):
    # Разделяем y_pred на части для классификации (y_pred_classification) и регрессии (y_pred_regression)
    y_pred = [y_pred[0], y_pred[1]]
    y_pred = tf.keras.layers.Concatenate()(y_pred)
    y_pred_classification, y_pred_regression = tf.split(y_pred, num_or_size_splits=2, axis=-1)


    y_true = [y_true[0], y_true[1]]
    y_true = tf.keras.layers.Concatenate()(y_true)
    # Разделяем y_true на части для классификации (y_true_classification) и регрессии (y_true_regression)
    y_true_classification, y_true_regression = tf.split(y_true, num_or_size_splits=2, axis=-1)

    # Компонент функции потерь для классификации (бинарная кросс-энтропия)
    # classification_loss = -1/num_classes * tf.reduce_sum(y_true_classification * tf.math.log(y_pred_classification + 1e-9) + (1 - y_true_classification) * tf.math.log(1 - y_pred_classification + 1e-9))

    # Компонент функции потерь для регрессии (среднеквадратичная ошибка)
    # regression_loss = 1/num_classes * tf.reduce_sum(tf.square(y_true_regression - y_pred_regression))

    # Общая функция потерь
    total_loss = -1/num_classes * tf.reduce_sum(y_true_classification * tf.math.log(y_pred_classification + 1e-9) + (1 - y_true_classification) * tf.math.log(1 - y_pred_classification + 1e-9)) + 1/num_classes * tf.reduce_sum(tf.square(y_true_regression - y_pred_regression))

    return total_loss


In [2]:
import json
from keras.layers import GlobalAveragePooling2D
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import Xception
from keras.layers import Conv2D, Dense
from keras.models import Model
import xml.etree.ElementTree as ET
from keras.utils import to_categorical, serialize_keras_object
import numpy as np


annotations_dir = 'annotation'
annotations = {}

# Функция для масштабирования координат в аннотациях
def scale_bbox(xmin, ymin, xmax, ymax, width_scale, height_scale):
    xmin_scaled = int(xmin * width_scale)
    ymin_scaled = int(ymin * height_scale)
    xmax_scaled = int(xmax * width_scale)
    ymax_scaled = int(ymax * height_scale)
    return xmin_scaled, ymin_scaled, xmax_scaled, ymax_scaled

# Создаем генератор данных для предобработки изображений
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.3  # Разделение данных на обучающий и валидационный наборы
)

for breed_folder in os.listdir(annotations_dir):
    if os.path.isdir(os.path.join(annotations_dir, breed_folder)):
        breed_annotations = {}
        for annotation_file in os.listdir(os.path.join(annotations_dir, breed_folder)):
            tree = ET.parse(os.path.join(annotations_dir, breed_folder, annotation_file))
            root = tree.getroot()
            class_name = root.find('object').find('name').text

            bndbox = root.find('object').find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # Путь к соответствующему изображению
            image_filename = os.path.splitext(annotation_file)[0]
            image_path = os.path.join('images', breed_folder, image_filename) + '.jpg'

            # Загружаем изображение и получаем его размер
            img = cv2.imread(image_path, cv2.IMREAD_COLOR)
            original_image_height, original_image_width, _ = img.shape

            # Рассчитываем коэффициенты масштабирования
            width_scale = 448 / original_image_width
            height_scale = 448 / original_image_height

            # Масштабируем координаты в аннотациях
            xmin, ymin, xmax, ymax = scale_bbox(xmin, ymin, xmax, ymax, width_scale, height_scale)

            breed_annotations[image_filename] = {'class_name': class_name, 'bbox': (xmin, ymin, xmax, ymax)}

        annotations[breed_folder] = breed_annotations

image_dir = 'images'

# Преобразование изображений с использованием генератора данных
batch_size = 32
generator = datagen.flow_from_directory(
    image_dir,
    target_size=(448, 448),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

images = []
labels = []

for _ in range(len(generator)):
    batch_images, batch_labels = generator.next()
    images.extend(batch_images)
    labels.extend(batch_labels)

# Преобразование меток в числовой формат (категориальное кодирование)
unique_labels = list(generator.class_indices.keys())
num_classes = len(unique_labels)
label_to_id = {label: i for i, label in enumerate(unique_labels)}
labels = [np.argmax(label) for label in labels]
labels = to_categorical(labels, num_classes=num_classes)
images = np.array(images)


# Загрузка предварительно обученной модели Xception
base_model = Xception(weights='imagenet', include_top=False)

# Входной тензор для одноканального изображения
input_image = tf.keras.Input(shape=(448, 448, 1))


pseudo_rgb_image = Conv2D(3, (1, 1))(input_image)

x = base_model(pseudo_rgb_image)
cam_x = base_model.get_layer('block13_sepconv2_act')(x)
x = GlobalAveragePooling2D()(cam_x)
old_weights = base_model.get_weights()

# 2xDense
classification = Dense(num_classes, activation='sigmoid')(x)
regression = Dense(num_classes, activation='linear')(x)

model = Model(inputs=input_image, outputs=[classification, regression])

# model.save('model_structure.h5', include_optimizer=False)


# Компилирование модели
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=custom_loss, metrics=[comp_classification_loss, comp_regression_loss])
model.fit(images, [labels, labels], batch_size=32, epochs=1, validation_split=0.3)

classification_config = serialize_keras_object(model.layers[-2])
classification_config_json = json.dumps(classification_config)

with open("classification_layer_structure.json", "w") as json_file:
    json_file.write(classification_config_json)
classification_layer_weights = model.layers[-2].get_weights()
regression_layer_weights = model.layers[-1].get_weights()

Found 589 images belonging to 3 classes.
13/13 [==============================] - 1851s 145s/step - loss: nan - dense_loss: 0.8249 - dense_1_loss: nan - dense_comp_classification_loss: 19.5932 - dense_comp_regression_loss: 6.8055 - dense_1_comp_classification_loss: nan - dense_1_comp_regression_loss: 16.5084 - val_loss: nan - val_dense_loss: 1.0995 - val_dense_1_loss: nan - val_dense_comp_classification_loss: 18.8436 - val_dense_comp_regression_loss: 6.6001 - val_dense_1_comp_classification_loss: nan - val_dense_1_comp_regression_loss: 30.2609


In [3]:
np.save('classification_weights_1.npy', classification_layer_weights[0])
np.save('classification_weights_2.npy', classification_layer_weights[1])

np.save('regression_weights_1.npy', regression_layer_weights[0])
np.save('regression_weights_2.npy', regression_layer_weights[1])

In [1]:
def preprocess_image(image_path, target_size=(448, 448)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, target_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)

    return img

In [7]:
import numpy as np
regression_layer_weights = [np.load('regression_weights_1.npy'), np.load('regression_weights_2.npy')]
classification_layer_weights = [np.load('classification_weights_1.npy'), np.load('classification_weights_2.npy')]
print(classification_layer_weights[1])

[-0.00272024 -0.00819132 -0.00259256]


In [7]:
from tensorflow.python.keras.models import load_model
from keras.layers import BatchNormalization
import numpy as np
import cv2
import matplotlib.pyplot as plt

regression_layer_weights = [np.load('regression_weights_1.npy'), np.load('regression_weights_2.npy')]
classification_layer_weights = [np.load('classification_weights_1.npy'), np.load('classification_weights_2.npy')]

# loaded_model = load_model('model_structure.h5',custom_objects={'BatchNormalization': BatchNormalization}, compile=False)
image = preprocess_image("images/n02085936-Maltese_dog/n02085936_233.jpg")
# plt.imshow(image)

In [26]:
from keras import Model
from keras.layers import Dense, deserialize
from keras.models import model_from_config, model_from_json
import tensorflow as tf
import json

input_image = tf.keras.Input(shape=(448, 448, 1))

with open("classification_layer_structure.json", "r") as json_file:
    loaded_classification_config_json = json_file.read()

loaded_classification_layer = deserialize(loaded_classification_config_json)


transformed_model = Model(inputs=input_image, outputs=loaded_classification_layer)

layer_output = transformed_model.predict(image)

# ram_image = np.dot(layer_output, regression_layer_weights)
# ram_image = np.squeeze(ram_image)

ValueError: Unknown layer: {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "units": 3, "activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [11]:
block5_output = model.get_layer('block5_sepconv2_act').output
block7_output = model.get_layer('block7_sepconv2_act').output
block9_output = model.get_layer('block9_sepconv2_act').output
block11_output = model.get_layer('block11_sepconv2_act').output

GAP_after_block5 = GlobalAveragePooling2D()(block5_output)
GAP_after_block7 = GlobalAveragePooling2D()(block7_output)
GAP_after_block9 = GlobalAveragePooling2D()(block9_output)
GAP_after_block11 = GlobalAveragePooling2D()(block11_output)

# block13_output = model.get_layer('block13_sepconv2_act').output

model_with_GAP = Model(inputs=model.input, outputs=[block5_output, GAP_after_block5, block7_output, GAP_after_block7, block9_output, GAP_after_block9, block11_output, GAP_after_block11])



ValueError: No such layer: block5_sepconv2_act. Existing layers are: ['input_2', 'conv2d_4', 'xception', 'block13_sepconv2_act', 'global_average_pooling2d', 'dense', 'dense_1'].